https://cryptography.io/en/stable/hazmat/primitives/cryptographic-hashes/

LIBSODIM cool aussi mais on peut plus diffcilement faire des .pem par ex

on peut plus facilement faire du openssl avec cryptography

In [19]:
# Pour fonction de hachage
import os
from cryptography.hazmat.primitives import hashes
# Pour courbe elliptique
from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives.asymmetric import ed25519
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers.aead import AESGCM


from nacl import bindings

In [20]:

# An application using X3DH must decide on several parameters:
HashFunction = hashes.SHA256()
curveSign = ed25519
curveExchange = x25519
info = "Myprotocol"

numberOPKB = 5

In [21]:
def DH():
    return None

def Sig():
    return None

def KDF():
    return None

In [22]:

#encoder des clés en bytes

def encode_public(key):
    bytes = key.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    return bytes

def encode_private(key):
    bytes = key.private_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PrivateFormat.Raw,
        encryption_algorithm=serialization.NoEncryption()
    )
    return bytes

# Fonction qui transforme une courbe elliptique de type ed privé en X privé
def ed_priv_to_X(ed_priv: ed25519.Ed25519PrivateKey):
    seed = ed_priv.private_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PrivateFormat.Raw,
        encryption_algorithm=serialization.NoEncryption()
    )
    pub = ed_priv.public_key().public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    sk_64 = seed + pub
    x_bytes = bindings.crypto_sign_ed25519_sk_to_curve25519(sk_64)
    return x25519.X25519PrivateKey.from_private_bytes(x_bytes)

# Fonctions qui transforme une courbe elliptique de type ed pub en X pub
def ed_pub_to_X(ed_pub: ed25519.Ed25519PublicKey):
    pub_bytes = ed_pub.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    x_bytes = bindings.crypto_sign_ed25519_pk_to_curve25519(pub_bytes)
    return x25519.X25519PublicKey.from_public_bytes(x_bytes)


In [23]:
class Alice():
    def __init__(self):
        self.__SK = None
        self.__PRIVATE_IKA = None
        self.PUBLIC_IKA = None
        self.__PRIVATE_EKA = None
        self.PUBLIC_EKA = None
        self.Bob_parameters = None
        self.BOB_PUBLIC_IKB = None
        self.BOB_PUBLIC_SPKB = None
        self.BOB_SIGN_SPKB = None
        self.BOB_PUBLIC_OPKB = None
        self.__AD = None
        self.canSendToBob = False

    def generation_general(self):
        # PRIVATE_IKA Alice’s identity key (long-term identity public key)
        self.__PRIVATE_IKA = curveSign.Ed25519PrivateKey.generate()
        # PUBLIC Alice’s identity key (long-term identity public key)
        self.PUBLIC_IKA = self.__PRIVATE_IKA.public_key()
    
    def EKA_renew(self):
        # During each protocol run, Alice generates a new ephemeral key pair with public key EKA.
        #EKA Alice’s ephemeral key
        self.__PRIVATE_EKA = curveExchange.X25519PrivateKey.generate()
        self.PUBLIC_EKA = self.__PRIVATE_EKA.public_key()
    
    def reception_Bob_parameters(self,values):
        tmp = values
        self.BOB_PUBLIC_IKB = tmp['PUBLIC_IKB']
        self.BOB_PUBLIC_SPKB = tmp['PUBLIC_SPKB']
        self.BOB_SIGN_SPKB = tmp['SIGN_SPKB']
        self.BOB_PUBLIC_OPKB = tmp['PUBLIC_OPKB']

        # Alice verifies the prekey signature and aborts the protocol if verification fails.
        public_bytes = self.BOB_PUBLIC_SPKB.public_bytes(
            encoding=serialization.Encoding.Raw,
            format=serialization.PublicFormat.Raw)
        try:
            # Leve une exception si problème
            verification = self.BOB_PUBLIC_IKB.verify(self.BOB_SIGN_SPKB,public_bytes)

            # Alice then generates an ephemeral key pair with public key EKA.
            self.EKA_renew()

            if self.BOB_PUBLIC_OPKB == None:

                # DH1 = DH(IKA, SPKB)
                key = ed_priv_to_X(self.__PRIVATE_IKA)
                DH1 = key.exchange(self.BOB_PUBLIC_SPKB)
                # DH2 = DH(EKA, IKB)
                key = ed_pub_to_X(self.BOB_PUBLIC_IKB)
                DH2 = self.__PRIVATE_EKA.exchange(key)
                # DH3 = DH(EKA, SPKB)
                DH3 = self.__PRIVATE_EKA.exchange(self.BOB_PUBLIC_SPKB)
                # SK = KDF(DH1 || DH2 || DH3)
                self.__SK = HKDF(
                    algorithm=hashes.SHA256(),
                    length=32,
                    salt=None,
                    info=b'test',
                ).derive(DH1+DH2+DH3)

                # After calculating SK, Alice deletes her ephemeral private key and the DH outputs.
                self.__PRIVATE_EKA = None
                DH1 = DH2 = DH3 = None

                # Alice then calculates an “associated data” byte sequence AD that contains
                # identity information for both parties:
                # AD = Encode(IKA) || Encode(IKB)

#                 (ATTENTION) :  Alice may optionally append additional information to AD, such as Alice and
# Bob’s usernames, certificates, or other identifying information.

                self.__AD = encode_public(self.PUBLIC_IKA) + encode_public(self.BOB_PUBLIC_IKB)
                
                # Alice then sends Bob an initial message :

                data = b'test first message to Bob'
                aesgcm = AESGCM(self.__SK)
                nonce = os.urandom(12)
                c = aesgcm.encrypt(nonce, data, self.__AD)
                

                first_message=[self.PUBLIC_IKA,self.PUBLIC_EKA,-1,c,nonce]
                self.canSendToBob = True

            else:
                 # DH1 = DH(IKA, SPKB)
                key = ed_priv_to_X(self.__PRIVATE_IKA)
                DH1 = key.exchange(self.BOB_PUBLIC_SPKB)
                # DH2 = DH(EKA, IKB)
                key = ed_pub_to_X(self.BOB_PUBLIC_IKB)
                DH2 = self.__PRIVATE_EKA.exchange(key)
                # DH3 = DH(EKA, SPKB)
                DH3 = self.__PRIVATE_EKA.exchange(self.BOB_PUBLIC_SPKB)
                # DH4 = DH(EKA, OPKB)
                DH4 = self.__PRIVATE_EKA.exchange(self.BOB_PUBLIC_OPKB[0])

                self.__SK = HKDF(
                    algorithm=hashes.SHA256(),
                    length=32,
                    salt=None,
                    info=b'test',
                ).derive(DH1+DH2+DH3+DH4)


                # After calculating SK, Alice deletes her ephemeral private key and the DH outputs.
                self.__PRIVATE_EKA = None
                # self.PUBLIC_EKA = None
                DH1 = DH2 = DH3 = DH4 = None


                # Alice then calculates an “associated data” byte sequence AD that contains
                # identity information for both parties:
                # AD = Encode(IKA) || Encode(IKB)

#                 (ATTENTION) :  Alice may optionally append additional information to AD, such as Alice and
# Bob’s usernames, certificates, or other identifying information.

                self.__AD = encode_public(self.PUBLIC_IKA) + encode_public(self.BOB_PUBLIC_IKB)
                
                
                # Alice then sends Bob an initial message :

                data = b'test first message to Bob'
                aesgcm = AESGCM(self.__SK)
                nonce = os.urandom(12)
                c = aesgcm.encrypt(nonce, data, self.__AD)
                
                
                first_message=[self.PUBLIC_IKA,self.PUBLIC_EKA,self.BOB_PUBLIC_OPKB[1],c,nonce]
                self.canSendToBob = True



        except InvalidSignature as e:
            first_message = None
            print("Signature invalide")

        return first_message


def sendToBob(message):
    
    return None

class Bob():

    def __init__(self):
        self.__SK = None
        self.__PRIVATE_IKB = None
        self.__PUBLIC_IKB = None
        self.__PRIVATE_SPKB = None
        self.__PUBLIC_SPKB = None
        self.__SIGN_SPKB = None
        self.__PUBLIC_OPKB_pairs = []
        self.__PRIVATE_OPKB_pairs = []
        self.ALICE_PUBLIC_IKA = None
        self.ALICE_PUBLIC_EKA = None

    def generation_general(self):
        # PRIVATE_IKB Bob’s identity key (long-term identity public key)
        self.__PRIVATE_IKB = curveSign.Ed25519PrivateKey.generate()
        # PUBLIC IKB Bob’s identity key (long-term identity public key)
        self.__PUBLIC_IKB = self.__PRIVATE_IKB.public_key()

        # (ATTENTION) A CHANGER PERIODIQUEMENT : pas fait
        #################

        # PRIVATE SPKB Bob’s signed prekey
        self.__PRIVATE_SPKB = curveExchange.X25519PrivateKey.generate()
        # PUBLIC SPKB Bob’s signed prekey
        self.__PUBLIC_SPKB = self.__PRIVATE_SPKB.public_key()

        #################

        # SIGNATURE of SPKB Bob’s signed prekey
        public_bytes = self.__PUBLIC_SPKB.public_bytes(
            encoding=serialization.Encoding.Raw,
            format=serialization.PublicFormat.Raw)
        self.__SIGN_SPKB = self.__PRIVATE_IKB.sign(public_bytes)

        # OPKB Bob’s one-time prekey
        cpt = 0
        for i in range(numberOPKB):
            PRIVATE_OPKB = curveExchange.X25519PrivateKey.generate()
            PUBLIC_OPKB = PRIVATE_OPKB.public_key()
            self.__PUBLIC_OPKB_pairs.append([PUBLIC_OPKB,cpt])
            self.__PRIVATE_OPKB_pairs.append([PRIVATE_OPKB,cpt])
            cpt = cpt + 1

    def send_bob_parameters(self):

        return {
            "PUBLIC_IKB": self.__PUBLIC_IKB,
            "PUBLIC_SPKB": self.__PUBLIC_SPKB,
            "SIGN_SPKB": self.__SIGN_SPKB,
            "PUBLIC_OPKB_pairs": self.__PUBLIC_OPKB_pairs
        }

    def messagefromAlice(self,message):
        self.ALICE_PUBLIC_IKA = message[0]
        self.ALICE_PUBLIC_EKA = message[1]
        BOB_PUBLIC_OPKB_ID = message[2]
        c = message[3]
        nonce = message[4]

        # Using these keys, Bob repeats the DH and KDF calculations from the previous
        # section to derive SK, and then deletes the DH values.

        if(BOB_PUBLIC_OPKB_ID == -1):
            
            # DH1 = DH(IKA, SPKB)
            key = ed_pub_to_X(self.ALICE_PUBLIC_IKA)
            DH1 = self.__PRIVATE_SPKB.exchange(key)
            # DH2 = DH(EKA, IKB)
            key = ed_priv_to_X(self.__PRIVATE_IKB)
            DH2 = key.exchange(self.ALICE_PUBLIC_EKA)
            # DH3 = DH(EKA, SPKB)
            DH3 = self.__PRIVATE_SPKB.exchange(self.ALICE_PUBLIC_EKA)


            self.__SK = HKDF(
                algorithm=hashes.SHA256(),
                length=32,
                salt=None,
                info=b'test',
            ).derive(DH1+DH2+DH3)


            # After calculating SK, Alice deletes her ephemeral private key and the DH outputs.
            DH1 = DH2 = DH3 = None

            self.AD = encode_public(self.ALICE_PUBLIC_IKA) + encode_public(self.__PUBLIC_IKB)


            aesgcm = AESGCM(self.__SK)
            message = aesgcm.decrypt(nonce, c, self.AD)
            print(message)
        else:
            # DH1 = DH(IKA, SPKB)
            key = ed_pub_to_X(self.ALICE_PUBLIC_IKA)
            DH1 = self.__PRIVATE_SPKB.exchange(key)
            # DH2 = DH(EKA, IKB)
            key = ed_priv_to_X(self.__PRIVATE_IKB)
            DH2 = key.exchange(self.ALICE_PUBLIC_EKA)
            # DH3 = DH(EKA, SPKB)
            DH3 = self.__PRIVATE_SPKB.exchange(self.ALICE_PUBLIC_EKA)
            # DH4 = DH(EKA, OPKB)
            for i in self.__PRIVATE_OPKB_pairs:
                if i[1] == BOB_PUBLIC_OPKB_ID:
                    key3 = i[0]
                    break
            DH4 = key3.exchange(self.ALICE_PUBLIC_EKA)


            self.__SK = HKDF(
                algorithm=hashes.SHA256(),
                length=32,
                salt=None,
                info=b'test',
            ).derive(DH1+DH2+DH3+DH4)


            # After calculating SK, Alice deletes her ephemeral private key and the DH outputs.
            DH1 = DH2 = DH3 = DH4 = None

            self.AD = encode_public(self.ALICE_PUBLIC_IKA) + encode_public(self.__PUBLIC_IKB)

            aesgcm = AESGCM(self.__SK)
            message = aesgcm.decrypt(nonce, c, self.AD)
            return message



class serveur():

    def __init__(self):
        self.BOB_PUBLIC_IKB = None,
        self.BOB_PUBLIC_SPKB = None,
        self.BOB_SIGN_SPKB = None,
        self.BOB_PUBLIC_OPKB_pairs = [],
        self.initialmessage = None
    
    def reception_Bob_parameters(self,values):
        tmp = values
        self.BOB_PUBLIC_IKB = tmp['PUBLIC_IKB']
        self.BOB_PUBLIC_SPKB = tmp['PUBLIC_SPKB']
        self.BOB_SIGN_SPKB = tmp['SIGN_SPKB']
        self.BOB_PUBLIC_OPKB_pairs = tmp['PUBLIC_OPKB_pairs']
    
    def send_bob_parameters(self):
        if self.BOB_PUBLIC_OPKB_pairs:
            val = self.BOB_PUBLIC_OPKB_pairs.pop(0)  # retourne et supprime le premier
        else:
            val = None
        return {
            "PUBLIC_IKB": self.BOB_PUBLIC_IKB,
            "PUBLIC_SPKB": self.BOB_PUBLIC_SPKB,
            "SIGN_SPKB": self.BOB_SIGN_SPKB,
            "PUBLIC_OPKB": val
        }
    def setInitialmessageAlice(self,m):
        self.initialmessage = m

    def getInitialmessageAlice(self):
        return self.initialmessage
    


In [ ]:
def X3DH(params): #algorithm has three phases:
    # 1. Bob publishes his identity key and prekeys to a server.

    # Generation from Bob
    User_Bob = Bob()
    User_Bob.generation_general()
    parameters_from_bob = User_Bob.send_bob_parameters()
    # Reception by serveur
    User_serveur = serveur()
    User_serveur.reception_Bob_parameters(parameters_from_bob)
    if(params):{ print(parameters_from_bob)}

    # 2. Alice fetches a “prekey bundle” from the server, and uses it to send an
    # initial message to Bob.
    User_Alice = Alice()
    User_Alice.generation_general()
    first_message = User_Alice.reception_Bob_parameters(User_serveur.send_bob_parameters())
    User_serveur.setInitialmessageAlice(first_message)
    
    # 3. Bob receives and processes Alice’s initial message.
    message = User_Bob.messagefromAlice(User_serveur.getInitialmessageAlice())
    print(message)


X3DH(0)


{'PUBLIC_IKB': <cryptography.hazmat.bindings._rust.openssl.ed25519.Ed25519PublicKey object at 0x10c5f72d0>, 'PUBLIC_SPKB': <cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f7e90>, 'SIGN_SPKB': b'\x94\xf7\x17\xd4h\x7f\x92I\xa47\xa3\\p\xd5K\x82\xb8\xbe\xe5\xcc*|2\x1a\xf9\xe5\x08\xe7\xa3_ \x1a\xe5\x87\xd9H\xf8\x90\xd8e\xc5\xac\x14\xa5&\xa2x!\x03\xf8e\x9e\x1d\xfe\xc7\x98\x8d&\xbf;\x11\xf0\xdd\x00', 'PUBLIC_OPKB_pairs': [[<cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f6ed0>, 0], [<cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f7870>, 1], [<cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f7d70>, 2], [<cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f7c50>, 3], [<cryptography.hazmat.bindings._rust.openssl.x25519.X25519PublicKey object at 0x10c5f7cb0>, 4]]}
b'test first message to Bob'
